In [ ]:
from lambeq import *
from quantum_pipeline import *
from utilities import *

In [2]:
#define atomic-types

N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [3]:
bobcat = BobcatParser(root_cats=('S', 'NP', 'PP', 'N'), verbose='progress')
iqp = IQPAnsatz({N: 1, S: 1, C: 1, P: 1, NP: 1, PP: 1}, n_layers = 2, n_single_qubit_params = 3)

In [4]:
pipeline = QuantumPipeline(bobcat, iqp)
#gps_labels, gps_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documenti/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/GPS.csv")
#cpn_labels, cpn_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documenti/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/CPN.csv")
#epurse_labels, epurse_circuits = pipeline.create_circuits_and_labels("/home/adriano22_/Documenti/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/ePurse.csv")

In [5]:
#save_data("bob_gps.txt", gps_labels, gps_circuits)
#save_data("bob_cpn.txt", cpn_labels, cpn_circuits)
#save_data("bob_epurse.txt", epurse_labels, epurse_circuits)

gps_labels, gps_circuits = load_data("bob_gps.txt")
cpn_labels, cpn_circuits = load_data("bob_cpn.txt")
epurse_labels, epurse_circuits = load_data("bob_epurse.txt")

In [7]:
test_circuits = epurse_circuits + gps_circuits
test_labels = epurse_labels + gps_labels

train_set, test_set = pipeline.create_dataset(cpn_circuits, cpn_labels), pipeline.create_dataset(test_circuits, test_labels)
model = pipeline.create_model(gps_circuits, cpn_circuits, epurse_circuits)


In [11]:
#Grid-Search setup
optimizers = [SPSAOptimizer, RotosolveOptimizer, NelderMeadOptimizer]
learning_rates = [5e-1, 5e-2, 5e-3]

train_set, test_set, eval_set = pipeline.create_dataset(gps_circuits, gps_labels), pipeline.create_dataset(cpn_circuits, cpn_labels), pipeline.create_dataset(epurse_circuits, epurse_labels)
model = pipeline.create_model(gps_circuits, cpn_circuits, epurse_circuits)

for optim in optimizers:
    for lr in learning_rates:
        print(f"Current combination:\n-Optimizer: {optim}\n-Learning rate: {lr}")
        pipeline.create_trainer(model = model, optimizer = optim, n_epochs = 100, a_hyp = lr, evaluate =  True)
        pipeline.train_and_evaluate(train_set, eval_set, 1, 1, None)

In [ ]:
pipeline.create_trainer(model = model, optimizer = RotosolveOptimizer, n_epochs = 100, a_hyp = 5e-1, evaluate =  True)
pipeline.train_and_evaluate(train_set, test_set, 1, 1, None)

In [ ]:
pipeline.create_trainer(model = model, optimizer = RotosolveOptimizer, n_epochs = 100, a_hyp = 5e-2, evaluate =  True)
pipeline.train_and_evaluate(train_set, test_set, 1, 1, None)

In [ ]:
pipeline.create_trainer(model = model, optimizer = RotosolveOptimizer, n_epochs = 100, a_hyp = 5e-3, evaluate =  True)
pipeline.train_and_evaluate(train_set, test_set, 1, 1, None)